In [44]:
#Importing the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, make_scorer 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model

In [3]:
!kaggle competitions download -c diamonds-datamad1020-rev

diamonds-datamad1020-rev.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!tar -xzvf diamonds-datamad1020-rev.zip

x sample_submission.csv
x test.csv
x train.csv


In [5]:
test_data = pd.read_csv('test.csv')
test_data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.33,Ideal,H,IF,61.9,55.0,4.44,4.42,2.74
1,1,0.41,Ideal,E,VS2,61.8,54.0,4.79,4.76,2.95
2,2,0.91,Very Good,E,SI2,62.5,59.0,6.16,6.23,3.87
3,3,0.42,Very Good,G,VS2,62.6,57.0,4.76,4.80,2.99
4,4,0.54,Ideal,G,IF,61.5,56.0,5.28,5.25,3.24


In [6]:
train_data = pd.read_csv('train.csv')
train_data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0.53,Very Good,G,SI1,63.4,54.0,5.09,5.13,3.24,7.057
1,1,0.41,Ideal,D,SI1,63.0,56.0,4.80,4.75,3.01,6.824
2,2,0.32,Ideal,I,VS2,61.6,56.0,4.37,4.39,2.70,6.107
3,3,0.31,Ideal,H,VVS2,61.2,56.0,4.34,4.37,2.66,6.390
4,4,1.35,Premium,J,VS2,60.5,56.0,7.19,7.12,4.33,8.741


In [7]:
#Let's check for collinearity issues
#Created a code to evaluate the collinearity between columns
corr_matrix = train_data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
to_drop

['x', 'y', 'z', 'price']

In [8]:
#Let's drop x, y, z columns. 
train_data.drop(['x','y','z'], axis = 1, inplace = True)
test_data.drop(['x','y','z'], axis = 1, inplace = True)

In [9]:
#Now, we will analyze numerical and categorical data - train data
numerical_cols = train_data.select_dtypes(include=np.number).columns.to_list()
categorical_cols = train_data.select_dtypes(exclude=np.number).columns.to_list()

In [10]:
#Now, we will analyze numerical and categorical data - test data
numerical_cols = test_data.select_dtypes(include=np.number).columns.to_list()
categorical_cols = test_data.select_dtypes(exclude=np.number).columns.to_list()

In [11]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 

train_data['cut'] = label_encoder.fit_transform(train_data['cut'])
train_data['color'] = label_encoder.fit_transform(train_data['color'])
train_data['clarity'] = label_encoder.fit_transform(train_data['clarity'])
train_data.head()

,id,carat,cut,color,clarity,depth,table,price
0,0,0.53,4,3,2,63.4,54.0,7.057
1,1,0.41,2,0,2,63.0,56.0,6.824
2,2,0.32,2,5,5,61.6,56.0,6.107
3,3,0.31,2,4,7,61.2,56.0,6.390
4,4,1.35,3,6,5,60.5,56.0,8.741


In [12]:
test_data['cut'] = label_encoder.fit_transform(test_data['cut'])
test_data['color'] = label_encoder.fit_transform(test_data['color'])
test_data['clarity'] = label_encoder.fit_transform(test_data['clarity'])
test_data.head()

,id,carat,cut,color,clarity,depth,table
0,0,0.33,2,4,1,61.9,55.0
1,1,0.41,2,1,5,61.8,54.0
2,2,0.91,4,1,3,62.5,59.0
3,3,0.42,4,3,5,62.6,57.0
4,4,0.54,2,3,1,61.5,56.0


In [13]:
X = train_data.drop(['price'],1)
y = train_data['price']

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

### KNeighbors Regression

In [22]:
#KNN
model = KNeighborsRegressor()
model.fit(X_train,y_train)

KNeighborsRegressor()

In [23]:
pred = model.predict(X_test)

In [24]:
mse = mean_squared_error(pred,y_test)
rmse = np.sqrt(mean_squared_error(pred,y_test))
mse,rmse

(1.195038639871462, 1.0931782287767453)

In [25]:
model.score(X_test,y_test)

-0.14902421672531063

### Tuning Parameters

In [26]:
ptmodel = KNeighborsRegressor()
ptrange = np.arange(1,11)
ptparams = ({'n_neighbors': ptrange})
gridsearch = GridSearchCV(ptmodel,ptparams,cv=10)

In [28]:
gridsearch.fit(X_train,y_train)
gridsearch.best_params_

{'n_neighbors': 10}

In [30]:
GSmodel = KNeighborsRegressor(n_neighbors=10)
GSmodel.fit(X_train,y_train)

KNeighborsRegressor(n_neighbors=10)

In [32]:
GSpred = GSmodel.predict(X_test)
GSmse = mean_squared_error(GSpred,y_test)
GSrmse = np.sqrt(mean_squared_error(GSpred,y_test))
GSmse

1.1168946745680386

In [34]:
GSmodel.score(X_train,y_train)

0.11763133401586723

In [35]:
prediction2 = GSmodel.predict(test_data)

In [36]:
len(prediction2)

13485

In [37]:
KNN_df = pd.DataFrame(prediction2, columns = ['price'])

In [38]:
KNN_df['id'] = KNN_df.index

In [39]:
KNN_df = KNN_df[['id', 'price']]

In [40]:
KNN_df.to_csv(r'KNN_df.csv', index=False)

### Lasso Regression

In [45]:
lr=LinearRegression()
lasso = linear_model.Lasso(alpha=0.1)
ridge = linear_model.Lasso(alpha=0.1)

In [46]:
lr.fit(X_train,y_train)
lasso.fit(X_train,y_train)
ridge.fit(X_train,y_train)

Lasso(alpha=0.1)

In [47]:
print("Linear Regression:",lr.score(X_test,y_test))
print("Lasso Regression:",lasso.score(X_test,y_test))
print("Ridge Regression:",ridge.score(X_test,y_test))

Linear Regression: 0.8688856079030053
Lasso Regression: 0.8017781222696652
Ridge Regression: 0.8017781222696652


In [50]:
lasso.score(X_train,y_train)

0.804409108011446

In [51]:
ridge.score(X_train,y_train)

0.804409108011446

In [52]:
prediction3 = lasso.predict(test_data)

In [53]:
Lasso_df = pd.DataFrame(prediction3, columns = ['price'])

In [54]:
Lasso_df['id'] = Lasso_df.index

In [55]:
Lasso_df = Lasso_df[['id', 'price']]

In [56]:
Lasso_df.to_csv(r'Lasso_df.csv', index=False)